In [1]:
from sklearn.dummy import DummyClassifier
import pandas as pd

In [5]:
train_data = pd.read_parquet("../data/processed/train_set_0.parquet.gzip")
test_data = pd.read_parquet("../data/processed/test_set_0.parquet.gzip")
X_train, y_train = train_data.iloc[:, :-1], train_data[["tracker"]]
X_test, y_test = test_data.iloc[:, :-1], test_data[["tracker"]]

Train dummy classifier

In [6]:
%%time
dummy_clf = DummyClassifier(strategy="uniform", random_state=10)
dummy_clf.fit(X_train, y_train)
dummy_clf.score(X_test, y_test)

CPU times: user 7.64 ms, sys: 2.01 ms, total: 9.65 ms
Wall time: 8.25 ms


0.5011040821198377